In [ ]:
import pandas as pd  

from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup   
import re 
import nltk

Загрузим тестовую и тренировочную выборки.

In [ ]:
train = pd.read_csv("../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("../input/word2vec-nlp-tutorial/testData.tsv.zip", header=0, delimiter="\t", quoting=3 )

In [ ]:
train

In [ ]:
test

Напишем функцию для обработки текста: функция убирает из текста html теги и знаки препинания, удаляет слова, невлияющие на обучение.

In [ ]:
def review_to_words( raw_review ):
    source_text = BeautifulSoup(raw_review).get_text()      
    letters_only = re.sub("[^a-zA-Z]", " ", source_text) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops]   
    
    return(" ".join(meaningful_words))  

Очистим данные тренировочной и тестовой выборки при помощи написанной функции.

In [ ]:
train_reviews_count = train["review"].size
clean_train_reviews = []
for i in range(0, train_reviews_count):
    clean_train_reviews.append(review_to_words(train["review"][i]))
    
test_reviews_count = test["review"].size
clean_test_reviews = []
for i in range(0, test_reviews_count):
    clean_test_reviews.append(review_to_words(test["review"][i]))

Получим вектора признаков из тренировочных и тестовых данных.

In [ ]:
vectorizer = CountVectorizer(analyzer = "word", 
                             tokenizer = None, 
                             preprocessor = None, 
                             stop_words = None, 
                             max_features = 20000) 

train_data_features = vectorizer.fit_transform(clean_train_reviews)
train_data_features = train_data_features.toarray()

test_data_feature = vectorizer.transform(clean_test_reviews)
test_data_feature = test_data_feature.toarray()

Проведем обучение модели при помощи рандомного леса (RandomForestClassifier) с количеством деревьев равным 300.

In [ ]:
clf = RandomForestClassifier(n_estimators = 300)
clf.fit(train_data_features, train.sentiment)

Получаем предсказание для тестовой выборки и формируем выходной файл.

In [ ]:
pred = clf.predict(test_data_feature)

output = pd.DataFrame(data={"id":test["id"], 
                            "sentiment":pred})

output.to_csv("submission.csv", index=False, quoting=3)